## **Efficient Programming in Python**
--- 
The topic of this notebook is efficient programming in Python. In real life most people when trying to solve a programming problem are less concerned with the efficiency of the solution than the actual solution. Our process is somewhat like this: 

 1. We receive the problem.
 2. We conceive a solution.
 3. We test the solution for correctness.
 4. The solution goes to production.
 5. We discover that the solution uses a lot of resources in production and/or is extremely slow.
 6. We refactor our solution to increase speed and to reduce resource usage.
 
In todays practice of agile software development I am not even sure that this is against best practice. In fact it seems almost that we should not spend too much time at point two, architecture an underated endeavour. My opinion would be to not go straight to a MVP, but to consider carefully how we built our solution. To take efficiency in both in time and space serious from the design up. Perhaps even before, at the choice of programming language. In case you program in Python, a quick way to improve the speed of your program is not to program in Python. If you move to a compiled language without garabage collection, you will increase the speed tremedously. Of course there are some risks involved, with that approach and since these notebooks are about Python we shall continue with efficiency in Python.

There are two levels of solutions to create efficient programs. At a deep level we can implement concurrency, program asynchronously or parallel. Of course, programming like that brings issues, at minimum a programmer needs to be very well versed in that technology to not make seriously costly mistakes. Though I want to discuss asynchronous programming (I find these subjects interesting), there is another manner with which to tackle the issues of inefficiency. These are higher level solutions, involving programming techniques such as:

 * Optimization 
 * Heuristics
 * Greedy algorithms 
 * Dynamic programming 
 * Using the correct data structures

These are still not always easy solutions, they need more thought, blow up your code base, and need practice. This notebook is about using these to create more efficient code. Below an example of simple direct search.

In [ ]:
from typing import Any

def lin_search(lst:list[Any],element:Any)->bool:
    for v in lst:
        if v == element:
            return True
    return False

lst = [*range(1,100_000_000)]
%timeit lin_search(lst,59_674_000)

Now the same problem coded up with binary search.

In [ ]:
def binary_search(lst:list[Any],element:Any)->bool:
    def search(lst:list[Any],element:Any,low:int,high:int)-> bool:
        if high == low: 
            return lst[low] == element
        mid = (high + low) // 2
        if lst[mid] == element: 
            return True
        elif lst[mid] > element:
            if low == mid: # the search has exhausted
                return False
            else:
                return search(lst,element,low,mid-1) 
        else:
            return search(lst,element,mid+1,high)
    
    if len(lst) == 0:
        return False
    else:
        return search(lst,element,0, len(lst)-1)

%timeit binary_search(lst,59_674_000)

#### **Analysis of algorithms; the shortest introduction ever**
As you can see, we have moved from second to microsecond, that is a million times faster. Where linear search is $O(n)$ binary search is $O(log(n))$ with respect to the length of the input n. Binary search is seriously faster, but it comes with a simple condition, the list needs to be sorted. Linear search works perfectly fine on unsorted list. 

I do not want to discuss search and sorting as such. What I am interested in is to show you several programming techniques with examples and give you an insight why these are more efficient. To do that I have to introduce you very quickly to the technique used to analyse algorithms. Firstly, when we analyse algorithms, we are foremostly interested in the running time of the algorithm, secondary is the space in memory the algorithm requires. I will use `%timeit` just to have an experimental way if showing code is faster or slower. Experimenting, though fun, is not the way to analyse algorithms. After all, speed would definitely be influenced on the hardware (mine is old and not that fast, yet beautiful and loved) but also on the input size. Waiting however, is boring, so we cannot use enormous input sizes.

Luckily enough for us some people already thought of the tools you need to analyse algorithms based upon on a high-level description of the algorithm. To do that we first consider all the things we think are primitive operations. We will define the following as primitive operations:

 * Assigning an identifier to an object
 * Determining the object associated with an identifier
 * Performing an arithmetic operation (e.g., adding two numbers)
 * Comparing two numbers
 * Accessing a single element of a Python list by index
 * Calling a function / method (excluding operations executed within the function)
 * Returning from a function / method

All these primitive operations are executed in constant time. With this in mind we can straight away see why binary search is so much faster than linear search. The latter needs to do nearly sixty million constant operations. The former basically divides the list in two, sees if the element wanted is bigger or smaller than the middle, moves over to whatever is the case and repeats the operation until the number is found or there are no more numbers.

{ `binary_search([50,000,000, ..., 100,000,000], 59,674,000)` }    
{ `binary_search([50,000,000, ..., 75,000,000], 59,674,000)` }    
{ `binary_search([50,000,000, ..., 62,500,000], 59,674,000)` }    
{ `binary_search([56,200,000, ..., 62,500,000], 59,674,000)` }    
{ `binary_search([59,300,000, ..., 62,500,000], 59,674,000)` }    
 
It takes 26 runs of binary search to find the target, linear search still will have 59,673,974 runs to go. This is a trade-off you quite often see; your code will get more verbose as the solution becomes more efficient.

When we analyse algorithms, it is easiest to assume worst case scenarios. It is often easy to identify a worst-case scenario. Furthermore, if you design algorithms to perform good (as good as can be achieved) for worst case, then obviously they will perform better in better scenarios. Big O notation gives us the worst-case scenario for the time an algorithm needs to compute a computation.

It is typical to use seven functions to describe speed of algorithms.

 1. $f(n) = c \rightarrow O(1)$ constant time is important because it signals the number of steps needed to do a basic operation, e.g., add two numbers together.
 2. $f(n)=log_b(n) \rightarrow O(log_2(n))$ Where the running time of an algorithm is a logarithmic to the input size, if n is 128, that $log_2(128)=7$  
 3. $f(n) = n \rightarrow O(n)$ linear time the running time is proportional to the input $O(128)=128$ 
 4. $f(n) = n \times log(n)  \rightarrow n \times O(log(n))$ NlogN is an intermediary function; This function grows a little more rapidly than the linear function and a lot less rapidly than the quadratic function. An algorithm that has a worst-case running time of NlogN is very much preferable over quadratic time. $ 7 \times 128 = 896$ while $128^2 = 16384$ or roughly 18 times as much with a very small input size.  
 5. $f(n) = n^2 \rightarrow O(n^2)$ Quadratic time, an algorithm in quadratic time would take a million CPU cycles if the input n is a mere thousand.
 5. $f(n) = n^3 \rightarrow O(n^3)$ Cubic time, an algorithm in cubic time would take a billion CPU cycles on the same input $n = 1000$
 7. $f(n) = 2^n \rightarrow O(2^n)$ Exponential time, an algorithm in quadratic time would take with input $n = 1000$, a number with 302 digits, cycles of the CPU to compute a solution. 

You might think that a programmatic solution with exponential time is per definition bad, but sometimes it is simply the best performance possible e.g., determine matrix chain multiplication. To determine the order with which we multiply matrices is exponential. In artificial intelligence several standard graph search algorithms are exponential in time/space. For instance, is breadth first search https://en.wikipedia.org/wiki/Breadth-first_search is exponential in space. 

That doesn't always matter the best-case scenario might be much better and the worsrt case scenario very rare. There might be optimizations on the algorithm we can perform. It is about these optimization techniques that this notebook is all about. For instance, when we went from linear search ($O(n)$) to binary search ($O(log(n))$) and increased the speed of our solution a million-fold by adding a few lines of code. Of course, the solution depends on both lists being sorted, and you should notice that in the best-case scenario linear search is faster, it would have `1` straight away. 

I do not want to say more, this is enough to understand when I say that algorithm or data structure is fast or slow and give you the big Oh. To do actual algorithm analysis requires a university course. Stanford does a MOOC about the subject, https://online.stanford.edu/courses/soe-ycsalgorithms1-algorithms-design-and-analysis-part-1 if you are interested.

Let's start with a true story.


#### **Croc and Pinky are going to rob the butcher (Optimization)**
There are great snackies inside the butcher which need to be rescued and eaten. Croc has considered it and gave the following values to the goodies in the butcher shop:
 * Tenderloin - 175
 * Shoulder of lamb - 90
 * Porkbelly - 20
 * Surloin - 50
 * Sausage - 10 
 * Cote du Boeuf - 200

Pinky, being the sensible one, has noted how much there is of each item by weight:
 * Tenderloin - 10
 * Shoulder of lamb - 9
 * Porkbelly - 4
 * Surloin - 2
 * Sausage - 1 
 * Cote du Boeuf - 20

Now they have a conundrum how the get the most value without exceeding the carrying limit of their innocent little baby arms and mouths (20kg the constraint). So they come to you for a solution. As you explained to Croc you can tackle this optimization problem in two ways, you can opt for the optimal solution, or you can go for the greedy one. To Croc this was nuff said, the greedy solution it had to be, as both Pinky and Croc were very greedy indeed.

Let's help this hungry salty and his dragon girlfriend!

In [ ]:
constraint = 20

from typing import NamedTuple

class Item(NamedTuple):
    name:str
    value:int
    weight:float
    
def build_items()->list[Item]:
    meat = ['Tenderloin', 'Shoulder of lamb', 'Porkbelly', 'Surloin', 'Sausage','Cote du Boeuf']
    values = [175,90,20,50,10,200]
    weights = [10,9,4,25,1,20]
    return [Item(meat[idx], values[idx], weights[idx]) for idx in range(len(meat))]

items = build_items()
 
# objective functions    
def value(item:Item)->int:
    return item.value

def weight_inverse(item:Item)->float:
    return 1 / item.weight

def density(item:Item)->float:
    return item.value / item.weight

# The greedy solution
def greedy(items:list[Item], constraint:int, key_func:object)->tuple[list[Item],int]:
    items = sorted(items, key=key_func, reverse=True)
    res = []
    total_value, total_weight= 0.0,0.0
    for item in items:
        if total_weight + item.weight <= constraint:
            res.append(item)
            total_weight += item.weight
            total_value += item.value
    return res,total_value

In [ ]:
greedy(items, constraint, value)

Croc surely will be happy, but Pinky will be mighty ticked off as Croc never shares his cote du boeuf. 

She demands another objective function!

In [ ]:
greedy(items, constraint, weight_inverse)

Now both Croc and Pinky are angry this is simply not acceptable loot, we need another objective function!

In [ ]:
greedy(items, constraint, density)

It seems the best optimizing function is a ratio of weight/value and not the value of the item itself.

#### **An optimal solution**
Unfortunately when Croc learned he couldn't take the cote du boeuf he threw an epic strop and he and is girlfriend Pinky where caught by the police, again...
To prevent any further episodes he now wants an optimal solution, the best possible in all circumstances.
A formalization of the problem is:
 1. Item is the pair (value,weight) they can take no more items than the constraint, of 20 kg.
 2. The set of available items snackies, a vector I of length n represents snackies. 
 
There is a vector V of length n with binary values, if `V[i]==1` than the snacky will be taken `V[i]==0` the snacky will be left behind, but under loud protest. Now the only thing we have to do is find vector V that maximizes $\sum_{i=0}^{n-1} V[i]*I[i].value$, subject to the constraint $\sum_{i=0}^{n-1} V[i]*I[i].weight \le constraint$  

We are creating a solution in three steps.
 1.  Create a powerset of the snackies set (all possible subsets including snackies itself and the empty set)

In [ ]:
# The itertools library is very rich with useful functions, this one prevents us from having to write a powerset function ourselves
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

pset = list(powerset(items))[1:]
len(pset)

2. We remove all sets that exceed the weight constraint.

In [ ]:
def rm_excess_weight(pset:list[tuple[Item]], constraint:int)->list[tuple[Item]]:
    res = []
    for items in pset:
        weight = 0.0
        for item in items:
            weight += item.weight
        if weight <= constraint:
            res.append(items)
    return res

pset = rm_excess_weight(pset,constraint)
len(pset)

 3. We select that set with best overall value.

In [ ]:
def snackies(pset:list[tuple[Item]])->tuple[Item]:
    snacks = ()
    best_value = 0.0
    for items in pset:
        value = 0.0
        for item in items:
            value += item.value
        if value > best_value:
            snacks = items
            best_value = value
    return snacks

In [ ]:
snackies(pset)

Unfortunately for Croc, it does make sense to leave the Cote du Boeuf behind. The best value is achieved by taking the above items. 
Fortunately dragons can spit fire and Croc heeled and the butcher was succesfully robbed!

#### **An analysis**
Here it ends for Croc and Pinky, but we should consider the above code.
We wil work backwards and start considering the snackies function. We have two iterables; pset and items, in the worst case scenario we would have to traverse both completely. Resulting in worst case scenario of $O(n^2)$
 
Now for rm_excess_weight this has only one iterable we need to traverse at worst this will take $O(n)$. 

Now let's look at powerset.

In [ ]:
l = set(powerset([1,2,3]))
l

In [ ]:
len(l)

In [ ]:
k = set(powerset([1,2,3,4]))
len(k)

I am sure you feel it coming but let's have a look on the powerset of a set with 5 elements.

In [ ]:
j = set(powerset([*range(1,6)]))
len(j)

Powerset is exponential in growth, as the input size grows the output size grows exponential $O(2^n)$.

Unfortunately there is nothing we can do. Taking the powerset of a set is always exponential relative to the size of the set. So the actual algorithmic complexity of the whole code is $O(2^n) + O(n^2) + O(n)= O(2^n)$

This might suprise you but as n get's bigger $O(2^n)$ quickly dwarfs $O(n^2)$ so the latter two become insignificant, so we only count the largest.

Our algorithm is not very fast, mostly due to the use of the powerset. If we would optimize this code, that is the place to start, how to prevent we use a powerset? 

The point of the story comes back to Herbert Simon, the father of Operational Research, said:

"Models making decisions that are good enough rather than laboriously calculating the optimal result, are a better description of human behaviour."

Programming solutions to problems often entails recognizing the pattern in the problem and applying specific programming styles to create a solution. Optimization is a very common problem for the developer to solve. For instance how many shoes a factory could produce given the material in stock, or what is the itinerary of traveling salesman?

An optimization problem exists of two parts:
 1. The objective function that is to be minimised or maximised. For instance the cost of a plane ticket between two cities when doing a search online, it needs to be the lowest possible in general.
 2. A set of constraints that must be honored. For instance a flight from Amsterdam to Liverpool should not take 6 hours and two flight changes. This set of constraints might include the empty set. 

These types of problems can often be formulated in a simple manner that lead to naturally computational solutions. They are problems that occur a lot in data intensive applications.

These problems are in general reducable to other well known problems, they follow the same pattern, problems with known solutions. Optimization problems can be solved with exhaustive enumerations algorithms, but because of the sheer amount of computation involved, more often than not greedy algorithms are used that deliver a fast sub-optimal but acceptable solution. In our case the greedy solution ran in $O(n)$ and yielded the same result when using density as optimizing function.

Finally quite often you will find that these problems can be solved recursively.


#### **Brute force**
The most common technique to solve a problem with computation is to use the brute computing power and force a solution, if any of course. The classic example is the one of pattern matching in a string, we are looking for:       
`abacab`    
in:    
`abacaabaccabacabaabb`

Now if I would have to come up with a solution I would definitely first get the solution by brute force.

In [ ]:
def brute_force_find(text:str, pattern:str)->bool:
    n, m = len(text), len(pattern)
    for i in range(n-m+1):
        k = 0
        for k in range(m):
            if text[i+k] == pattern[k]:
                k += 1
        if k == m:
            return True
    return False
brute_force_find('abacaabaccabacabaabb', 'abacab')    

#### **Heuristics**
This algorithm is $O(nm)$ we run two loops the outer (n-m+1) times if the size of n is much larger then m, the latter becomes insignificant so we say we run the outer loop n times. The innerloop is clear we run that m times. 

Now we have our brute force solution we can improve it. We could try to use some heuristics. A heuristic is any approach to problem solving or self-discovery that employs a practical method that is not guaranteed to be optimal, perfect, or rational, but is nevertheless sufficient for reaching an immediate, short-term goal or approximation (definition from Wikipedia). The easiest example of an heuristic is a direct line between two cities as a measure of distance. We all know that the distance between cities is usually not travelled in direct line, but it gives us a workable idea, to compare the distances between different cities and most of the time using our heuristic will give us an accurate sense of what city is quicker to travel to. 

Given our pattern `abacab` we can jump immidiately to the 6th position in `abacaabaccabacabaabb`, compare the last element of our pattern to that position; if $ a \ne b$, we can discount all that came until then. This approach is known as the looking glass heuristic. 

However great that is on it's own it gives only a slight improvement, but it allows me to introduce a second heuristic, the character jump heuristic. Say that our text looks like `abacadbaccabacabaabb` now if I check the 6th character I find a `d`, `d` is not in the pattern, so I can easily conclude that the pattern will not partially be in the charachters I just jumped. Let me again jump six I am now at `abacadbacca[b]acabaabb` `b` is a character in our pattern `abacab` so I can not just jump another six. When a match is found for that last character, the algorithm continues by trying to extend the match with the second-to-last character of the pattern from where it currently is aligned with the text. I move three positions to the left comparing characters `abacadba[c]cabacabaabb`, than I meet a `c` where I suposed to get an `a`. From `abacadba[c]cabacabaabb` I jump again six positions and I arrive at `abacadbaccabac[a]baabb`. The character `a`  is part of the pattern. Now I start shifting the pattern to the left until I meet the first character in the pattern that matches `a`.

`abacadbaccabac[a]baabb`    
`----------abac[a]b-----`    

The direction of the jump is dependent if we find the character before or after in the pattern. If I find the charachter in the pattern after the one I was expecting I jump to the right, otherwise I jump to the left. In our example I jump to the left. I can now simply check left and right for our pattern and find the match. 

This algorithm is called the Boyer-Moore algorithm. Though this algorithm is also $O(nm)$ in reality it will perform much better than our previous solution.

In [ ]:
from typing import Union

def boyer_moore_find(text:str, pattern:str)->Union[bool, int]:
    '''algorithm to find the lowest substring, return the lowest index where the substring starts
       or False if there is no substring that matches the pattern
       inspired by Boyer Moore'''
    n, m = len(text), len(pattern)
    if m == 0: return 0 # an empty pattern is always a substring 
    last = dict() # build ’last’ dictionary
    for k in range(m):
        last[pattern[k]] = k # later occurrence overwrites
    # align end of pattern at index m-1 of text
    i = m - 1
    k = m - 1
    while i < n:
        if text[i] == pattern[k]: # a matching character
            if k == 0:
                return i # pattern begins at index i of text
            else:
                i -= 1
                k -= 1 
        else:
            j = last.get(text[i], -1) # last(text[i]) is -1 if not found
            i += m - min(k, j + 1) # case analysis for jump step
            k = m - 1 # restart at end of pattern
    return False
boyer_moore_find('abacaabaccabacabaabb', 'abacab')


The Boyer Moore algorithm can improved upon, as Knuth, Morris, and Pratt did with their algorithm, which performs the operation in $O(m+n)$. You should try to understand it and where it improved Boyer Moore, see https://en.wikipedia.org/wiki/Knuth%E2%80%93Morris%E2%80%93Pratt_algorithm. 

I am not really interested in the result as much, as I am in the process. After the fitst iteration we have a straight forward brute force solution to a problem. In the second iteration we look at the properties of our problem and came with a smarter solution by introducing some heuristics (well Boyer and Moore did). In a next iteration we could improve it futher. This is often how code gets better. However you would need to incorporate this type of process in your programming proces and you need to accept that code refactoring is not criticism on your code.  

#### **Greedy algorithms**
condider the following matrix. 

$\begin{matrix}
  a & b & c \\
  d & e & f \\
  g & h & i
 \end{matrix}$
 
Say you need to travel from a to i, you have four moves available; Left, Right, Up, and Down. Easy done in four steps, but now I give a value to all elements.

$\begin{matrix}
  (a,0) & (b,1) & (c,1)\\
  (d,2) & (e,4) & (f,6)\\
  (g,2) & (h,2) & (i,1)
 \end{matrix}$
 
End I will ask you to find me the cheapest path. Easy you say a-d-g-h-i, 7 points. 

We make it a bit more difficult, we let you write a program to do this, easy you think, with the constraint that you will have to find us the best path possible in in only one go and that you can only know the cost of the nodes you are about to travel to, not the others, you can't go to a node you have already visited. 

The best a computer can do is make the optimal choice at every junction it needs to make a choice. Starting at a, the computer can go down at a cost of `2`, or can go right at the cost of `1`. As we are trying to minimize the cost and we have no more information available, there is only one choice that is rational; we go to `b`. From b we have again to options; `e` at a cost of `4`, or `c` at cost of `1`. The algorithm must optimize, it goes to `c`. From there we have only one choice we go to `f` at a cost of `6`. By now our cost are higher than the optimal cost. Yet this is the best we can do given our limited knowledge of the entire domain.

This is an example of a greedy algorithm, always making the optimal local choice. This problem however does not have the greedy-choice property; the global optimal condition cannot be reached by a series of locally optimal choices, choices that are each the current best from among the possibilities available at the time,
starting from a well-defined starting condition `a` in our case.

There are problems that have the greedy-choice property, the most famous is the huffman code, which allows you to compress text without loosing information, see https://en.wikipedia.org/wiki/Huffman_coding. But even without having the greedy-choice property a greedy algorithm might create acceptable results.

The first solution to Croc and Pinky's problem was greedy, where we used three heuristics to get a different outcome.

#### **Dynamic programming**
Dynamic programming that is a technique used to find the optimal solution to the initial problem by solving its sub-problems and combining their solutions. Recursion is a technique with which we can break down such problems into smaller subproblems, solve them and put the end result back together. 

Below is an example of tail recursion, being used to calculate the product of a list of numbers.

In [ ]:
def product(numbers:list[int])->int:
    if len(numbers) == 0:
        return 1
    n,*ns = numbers
    return n * product(ns)

%timeit product([1,2,3,4,5])


We can make a quick improvement (both in memory use and speed) by using an accumulator, which stores the intermediate results, and replacing the recursive call by a while loop. 

In [ ]:
def aproduct(numbers:list[int])->int:
    acc = 1
    for number in numbers:
        acc *= numbers.pop()
    return acc
    
%timeit aproduct([1,2,3,4,5]) 

#### **Fibonacci** 
As I already said many of the problems we are talking about have a recursive nature. The classic example if the Fibonacci function below.
This code is extremely inefficient because of the manner we recalculate all intermediate results when we recurse. Requestion only the fortieth number creates a very waiting time.  

In [ ]:
def fibonacci(n:int)->int:
    if n == 0:
        return 0 
    if n == 1:
        return 1
    return fibonacci(n-1) + fibonacci(n-2) 

In [ ]:
%time fibonacci(10)

In [ ]:
%time fibonacci(20)

In [ ]:
%time fibonacci(40)

#### **Memoization**
The technique of adding a memory structure, for instance a look-up-table, to your recursive code is called memoization. If we encounter a sub-problem that we have encountered before , we do not calculate its solution again, we read it from memory instead. Reading from memory is a constant time operation, therefore we gain speed.

Working with memoization is easy for we can still use the recursive nature of the problem.
 

In [ ]:
table = {}

def fibonacci(n:int)->int:
    if n == 0:
        return 0
    if n == 1:
        return 1 
    elif n in table:
        return table[n]
    else:
        table[n] = fibonacci(n-1) + fibonacci(n-2)
        return table[n]

%time fibonacci(40)    

Unfortunately Python has a very limited recursion depth, because it does not do any optimization of recursion.

This makes using memoization in Python not a realistic option.

In [ ]:
fibonacci(4000)

In [ ]:
import sys
sys.getrecursionlimit()

We can enlarge the recursion limit to say 100_000

In [ ]:
sys.setrecursionlimit(100_000)
fibonacci(4000)

#### **Tabulation**
However, while 100_000 might seem like a big number it really isn't. In Python recursion is than also not the manner uses for repetition, it uses iteration. For problems where recursion is natural, like product, sum or factorial or so, programming without recursion requires additional coding techniques. In an imperative language like Python the technique to use is tabulation.

Tabulation starts with the base case(s), finds the optimal solutions to the problem, whose immediate sub-problem is the base case. After which it goes a level higher, combines the solutions it previously obtained and construct the optimal solutions to more complex problems. The most important for a Python programmer is that we can leave recursion and use iteration. The biggest disadvantage of using tabulation is that it is not always easy to see how we can replace a recursive pattern with tabulation.

As you will see when running the next cell there is near enough no difference in running times between the accumulated product and the tabulated product algorithms. Both have an $O(n)$ running time. `aproduct` does n function calls, while `tproduct` loops over the list `ns` n-1 times. 

In [ ]:
def fibonacci(n:int)->int:
    '''tabulation example'''
    tab = [None]*(n+2) # Just for the base case I need the table to be two bigger
    tab[0] = 0
    tab[1] = 1
    for idx in range(2,n+1):
        tab[idx] = tab[idx-1] + tab[idx-2]
    return tab[n]
%time fibonacci(4000)

#### **Knapsack problem**
I admit it, I was lazy while writing the optimal solution to Croc's butcher problem. I used a powerset and than filtered out the unwanted results. Getting me 14 usuable sets out of an original 63. As long as I can guarantee that the input is small in size that wouldn't matter, but we should have a fast algorithm for any size input.

The obvious solution is not to use a powerset, but to just use those combinations whose combined weight is less than the constraint. We would completely be able to forgo any filtering if we create a sorted result. This is a well known dynamic programming problem, called the knapsack problem: Given a bag with capacity W and a list of items along with their weights and value associated with them. The task is to fill the bag in such a manner that the max value is returned, without exceeding W. 

The pseudo is:   

for each item 'i' starting from the end: // We recurse after all   
$\rightarrow$ create a new set that includes item 'i' iff the total weight < the constraint, recursively process the remaining capacity and items.    
$\rightarrow$ create a new set WITHOUT item 'i', and recursively process the remaining items 
 
return max(set1, set2) 

This is the standard recursive code, like or code it runs in  $O(2^n)$

In [ ]:
def knapsack(values, weights, capacity):
    
    def recursion(values, weights, capacity, current_index):
        # base cases
        if capacity <= 0 or current_index >= len(values):
            return 0

        result1 = 0

        if weights[current_index] <= capacity:
            result1 = values[current_index] + recursion(values, weights, capacity - weights[current_index], current_index + 1)

        result2 = recursion(values, weights, capacity, current_index + 1)

        return max(result1, result2)
    
    return recursion(values, weights, capacity, 0)


knapsack([1, 6, 10, 16], [1, 2, 3, 5], 7)

#### **Memoization**
trace:
`knapsack([1, 6, 10, 16], [1, 2, 3, 5], 7)` $\rightarrow$ `recursion([1, 6, 10, 16], [1, 2, 3, 5], 7, 0)`        
`max(result1 = 1 + recursion([1, 6, 10, 16], [1, 2, 3, 5], 6, 1), result2 = recursion([1, 6, 10, 16], [1, 2, 3, 5], 6, 1))`  6 + recursive call $\rightarrow$ 
max(`10 + recursion([1, 6, 10, 16], [1, 2, 3, 5], 3, 3), recursion([1, 6, 10, 16], [1, 2, 3, 5], 6, 3)`) = max(10,16) $\rightarrow$ 6 + 16 = 22 

This is a very short trace but I think you will understand. Don't worry if that takes time, it is an old problem (100+ years) most people including me had to take their sweet time to grasp the details.

The above solution is exponential in time $O(2^n)$ due to the number of calls with overlapping subcalls. For instance if I call: `knapsack([1, 6, 10, 16], [1, 2, 3, 5], 7)`

I will call `recursion([1, 6, 10, 16], [1, 2, 3, 5], 7)` which will fork in:

result1 = 1 + `recursion([1, 6, 10, 16], [1, 2, 3, 5], 6, 1)` 

result2 = `recursion([1, 6, 10, 16], [1, 2, 3, 5], 6, 1)`

Both calls are basically the calculation, I now perform twice, and that seeps through the entire algorithm.

If we get rid of the redunancy we can improve the speed to $O(C \times  N)$ 

We apply the same technique we have done before, we create a memory structure and instead recalculation partial results we just look them up. Consider the memory structure carefully. We need to keep two things in mind, we need to know the constraint and the weight that will give us the value. For instance if our capacity > 0 and our weight = 1 than our value be always 1. If on the other hand our capacity is >= 3, then our weight can be 7 and 10. We can capture that in a matrix. 

weight capacity 0  1  2  3  4  5  6  7     
0 --------value 0  0  0  0  0  0  0  0    
1 --------value 0  1  1  1  1  1  1  1    
2 --------value 0  0  6  7  7  7  7  7      
3 --------value 0  0  0  10 17 17 17 17     
5 --------value 0  0  0  0  0  16 17 22     



In [ ]:
def knapsack(values, weights, capacity):
    matrix = [[None for x in range(capacity+1)] for y in range(len(weights))]
    
    def recursion(matrix, values, weights, capacity, current_index):
        # base cases
        if capacity <= 0 or current_index >= len(values):
            return 0

        if matrix[current_index][capacity] is not None:
            return matrix[current_index][capacity]

        # recursive call after choosing the element at the currentIndex
        # if the weight of the element at currentIndex exceeds the capacity, we
        # shouldn't process this
        result1 = 0
        
        if weights[current_index] <= capacity:
            result1 = values[current_index] + recursion(matrix, values, weights, capacity - weights[current_index], current_index + 1)
        
        result2 = recursion(matrix, values, weights, capacity, current_index + 1) 
        
        matrix[current_index][capacity] = max(result1, result2)
        return matrix[current_index][capacity]
        
    
    return recursion(matrix, values, weights, capacity, 0)


knapsack([1, 6, 10, 16], [1, 2, 3, 5], 7)

We can apply the same coding technique to Croc's problem.

In [ ]:
from typing import NamedTuple

class Item(NamedTuple):
    name:str
    value:int
    weight:int
    
def build_items()->list[Item]:
    meat = ['Tenderloin', 'Shoulder of lamb', 'Porkbelly', 'Surloin', 'Sausage','Cote du Boeuf']
    values = [175,90,20,50,10,200]
    weights = [10,9,4,25,1,20]
    return [Item(meat[idx], values[idx], weights[idx]) for idx in range(len(meat))]

items = build_items()

constraint = 20

def snackies(items:list[Item], constraint:int)->int:
    matrix = [[None for x in range(constraint+1)] for y in range(len(items))]
    
    def recursion(matrix, items, constraint, current_index):
        # base cases
        if constraint <= 0 or current_index >= len(items):
            return 0

        if matrix[current_index][constraint] is not None: # we start at the back as per our pseudo
            return matrix[current_index][constraint]

        result1 = 0
        
        if items[current_index].weight <= constraint:
            result1 = items[current_index].value + recursion(matrix, items, constraint - items[current_index].weight, current_index + 1)
        
        result2 = recursion(matrix, items, constraint, current_index + 1) 
        
        matrix[current_index][constraint] = max(result1, result2)
        return matrix[current_index][constraint]
        
    
    return recursion(matrix, items, constraint, 0)

snackies(items, constraint)


#### **Efficient datastructures**
There are many more ways to solve dynamic programming, which usually boils down to finding the best result in a search space. You could for instance use a genetic algorithm. However diving into search algorithms for bound and unbound space really is another topic. What we should consider is this, if we add a memory structures to an algorithm, which data structures should we consider using? For certain data structures are simply faster and/or take less space than others. For instance the built-in type tuple is faster and takes less space than the built-in type list. See the stack overflow answer by Raymond Hettinger https://stackoverflow.com/questions/68630/are-tuples-more-efficient-than-lists-in-python/22140115#22140115. 

However both lists and tuples are hardly the fastest most efficient datastructure out there. They come with another distinct disadvantage, how to search an item in the list or tuple that is not sorted? That would take $O(N)$ where N is length of the list/tuple. Of course when we sort the list, then we can use a fast searching algorithm, but sorting comes with a price. 

Instead of considering how we could quickly search for a value in some sort of container, we could consider how to fill the container in such a manner that we could easily retrieve the right item. For instance if we kept track of the position of an element in a list, retrieving it would be a constant operation. Of course keeping track of positions will require additional book keeping, which also comes at a price. There are data structures that avoid this additional book keeping due to their internal organisation.

The classic examples are binary search tree & heaps.

`pip install binarytree`

In [4]:
from binarytree import tree, bst

my_tree = bst(height=3, is_perfect=True)

print(my_tree)


        ______7_______
       /              \
    __3__           ___11___
   /     \         /        \
  1       5       9         _13
 / \     / \     / \       /   \
0   2   4   6   8   10    12    14



A binary tree is an ordered tree with the following properties:
1. Every node has at most two children.
2. Each child node is labeled as being either a left child or a right child.
3. A left child precedes a right child in the order of children of a node.

We can define such a tree recursively:

A binary tree is either empty or consists of:
• A node r, called the root of T, that stores an element
• A binary tree (possibly empty), called the left subtree of T
• A binary tree (possibly empty), called the right subtree of T

A tree class would look like:

In [7]:
from typing import Optional 

class Position:
    ...

class Tree:

    def left(p:Position) -> Optional[Position]:
        '''Return the position that represents the left child of p, or None if p has no left child.'''
        ...
        
    def right(p:Position) -> Optional[Position]:
        '''Return the position that represents the right child of p, or None if p has no right child.'''
        ...
    def sibling(p:Position) -> Optional[Position]:
        '''Return the position that represents the sibling of p,or None if p has no sibling.'''
        ...